<a href="https://colab.research.google.com/github/Mallmulk/PemrogVisualWinform_P4/blob/main/Program_Klasifikasi_Daun_Tanaman_untuk_Identifikasi_Jenis_Tumbuhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os


Load dan Preprocessing Dataset

In [11]:
train_dir = '/content/dataset/train'
val_dir = '//content/dataset/val'
test_dir = '/content/dataset/test'


# Normalisasi nilai piksel (0-1)
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    'dataset/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = datagen.flow_from_directory(
    'dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)



Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Membangun Model (Transfer Learning MobileNetV2)

In [7]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze layer awal

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Training Model

In [8]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0

Evaluasi Model

In [9]:
test_data = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

loss, acc = model.evaluate(test_data)
print(f"Akurasi Model: {acc*100:.2f}%")

# Confusion Matrix
pred = np.argmax(model.predict(test_data), axis=1)
print(classification_report(test_data.classes, pred, target_names=list(test_data.class_indices.keys())))


Found 0 images belonging to 0 classes.


ValueError: The PyDataset has length 0

Visualisasi Hasil

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Akurasi Model')
plt.show()


Simpan Model

In [ ]:
model.save('model_klasifikasi_daun.h5')
